# **Customer Shopping Behavior**

## 1. Introduction
This project analyzes customer shopping behavior to understand how demographic and product-related factors influence spending patterns, product preferences, and subscription behavior.


## 2. Import Libraries

In [15]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

## 3. Load Dataset

In [5]:
df = pd.read_csv('../Data/customer_shopping_behavior.csv')

df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


## 4. Explore Data

In [6]:
df.shape

(3900, 18)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [8]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [9]:
# Converting uppercase columns to lowercase, change column name and replace space

df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.rename(columns={'purchase_amount_(usd)': 'purchase_amount'},inplace=True)
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [10]:
df.isnull().sum()

customer_id                0
age                        0
gender                     0
item_purchased             0
category                   0
purchase_amount            0
location                   0
size                       0
color                      0
season                     0
review_rating             37
subscription_status        0
shipping_type              0
discount_applied           0
promo_code_used            0
previous_purchases         0
payment_method             0
frequency_of_purchases     0
dtype: int64

## 5. Handle missing values and add new column

In [11]:
df['review_rating'] = df['review_rating'].fillna(df['review_rating'].median())


In [12]:
# Create age_group column

def age_group(age): 
    if 10 <= age <=20: return 'Children'
    elif 21 <= age <= 30: return 'Young'
    elif 31 <= age <=50: return 'Min-senior'
    else: return 'Senior'

df['age_group'] = df['age'].apply(age_group)

## 6. EDA and Statistics

#### Distribution of age
- H₀ (Null Hypothesis): Age is normally distributed
- H₁ (Alternative Hypothesis): Age is not normally distributed

In [13]:

stats, p = stats.shapiro(df['age'])

if p < 0.05:
    print('Age is not normally distributed. (Reject H0)')
else:
    print('Age is normally distributed. (Fail to reject H0)')

Age is not normally distributed. (Reject H0)


#### Distribution of purchase amount
- H₀ (Null Hypothesis): Purchase amount is normally distributed
- H₁ (Alternative Hypothesis): Purchase amount is not normally distributed

In [16]:
stats , p = stats.shapiro(df['purchase_amount'])

if p < 0.05:
    print('Purchase amount is not normally distriuted. (Reject H0)')
else:
    print('Purchase amount is normally distributed. (Fail to reject H0)')

Purchase amount is not normally distriuted. (Reject H0)


- H0 (Null Hypothesis): There is no relationship between gender and subscription
- H1 (Alternative Hypothesis): There is relationship between gender and subscription

In [17]:
con_table = pd.crosstab(df['gender'],df['subscription_status'])
con_table

subscription_status,No,Yes
gender,,
Female,1248,0
Male,1599,1053


In [ ]:

s, p, dof, e = stats.chi2_contingency(con_table)

if p < 0.05:
    print("There is a relationship between Gender and Subscription (Reject H0)")
else:
    print('There is no relationship between Gender and Subscription (Fail to reject H0)')

There is a relationship between Gender and Subscription (Reject H0)


In [20]:
# Average spending by gender, category and age_group

df.groupby('gender')['purchase_amount'].mean().sort_values(ascending=False)

gender
Female    60.249199
Male      59.536199
Name: purchase_amount, dtype: float64

In [21]:
df.groupby('category')['purchase_amount'].mean().sort_values(ascending=False)

category
Footwear       60.255426
Clothing       60.025331
Accessories    59.838710
Outerwear      57.172840
Name: purchase_amount, dtype: float64

In [22]:
df.groupby('age_group')['purchase_amount'].mean().sort_values(ascending=False)

age_group
Young         60.753053
Senior        59.945799
Min-senior    59.201356
Children      58.981132
Name: purchase_amount, dtype: float64

In [23]:
# Correlation between age and purchase_amount

corr = df['age'].corr(df['purchase_amount'],method='spearman')
corr

-0.010444519559151589

## 7. Connecting Python script to PostgreSQL

In [25]:
load_dotenv()

username = os.getenv('username')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')


table_name = 'customers'
df.to_sql(table_name,engine,if_exists='replace',index=False)
print('Connected')

Connected
